## The correct one 

In [1]:
from apscheduler.schedulers.background import BackgroundScheduler

In [2]:
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy import text

# Connection details for source and target databases
server_source = 'MENNAS-LAPTOP\\MSSQLSERVER22'
database_source = 'AdventureWorks2019'
username_source = 'sa'
password_source = 'sa123456'
driver = 'ODBC Driver 17 for SQL Server'

server_target = 'MENNAS-LAPTOP\\MSSQLSERVER22'
database_target = 'HR'
username_target = 'sa'
password_target = 'sa123456'

# Create engine for source and target databases
connection_string_source = (
    f'mssql+pyodbc://{username_source}:{password_source}@{server_source}/{database_source}'
    f'?driver={driver.replace(" ", "+")}'
)
engine_source = create_engine(connection_string_source, echo=False)

connection_string_target = (
    f'mssql+pyodbc://{username_target}:{password_target}@{server_target}/{database_target}'
    f'?driver={driver.replace(" ", "+")}'
)
engine_target = create_engine(connection_string_target, echo=False)


In [3]:

import traceback
import logging
from datetime import date,timedelta,datetime
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
logger = logging.getLogger(__name__)

import getpass


In [4]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

def send_mail_error(
    sender="osamamenna1511@gmail.com",  # put sender email here 
    password='lwrx jrtd uhjm cjvt',  # add a comma here
    to_list=["menna1eminshawy@gmail.com"],  # receiver list here
    error=None,
    subject="Error AD",
):
    msg = MIMEMultipart()
    msg["From"] = sender
    msg["To"] = ", ".join(to_list)
    msg["Subject"] = subject
    message = f"""<html><body>{error}</body></html>"""
    msg.attach(MIMEText(message, "html"))
    
    # sends email
    smtpserver = smtplib.SMTP("smtp.gmail.com", 587)
    smtpserver.starttls()
    smtpserver.login(sender, password)  # use the 'sender' variable instead of hardcoding
    smtpserver.sendmail(msg["From"], to_list, msg.as_string())
    smtpserver.quit()


In [5]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

def send_mail_success(
    sender="osamamenna1511@gmail.com",  # put sender email here 
    password='lwrx jrtd uhjm cjvt',  # use your app-specific password here
    to_list=["menna1eminshawy@gmail.com"],  # receiver list here
    message="Data transformation completed successfully.",
    subject="Data Transformation Success",
):
    msg = MIMEMultipart()
    msg["From"] = sender
    msg["To"] = ", ".join(to_list)
    msg["Subject"] = subject
    html_message = f"""<html><body>{message}</body></html>"""
    msg.attach(MIMEText(html_message, "html"))
    
    try:
        # sends email using IP address as a temporary workaround
        smtpserver = smtplib.SMTP("74.125.200.108", 587)  # Gmail SMTP IP address
        smtpserver.starttls()
        smtpserver.login(sender, password)
        smtpserver.sendmail(msg["From"], to_list, msg.as_string())
        smtpserver.quit()
        print("Success email sent successfully.")
    except Exception as e:
        print(f"Failed to send success email: {e}")

# Example usage after data transformation is done
send_mail_success(
    message="The data transformation process has been completed successfully and is ready for further steps."
)


Success email sent successfully.


In [6]:
def disable_foreign_keys(engine):
    with engine.connect() as connection:
        query = """
        SELECT 
            ss.name AS SchemaName,
            tp.name AS TableName
        FROM 
    sys.foreign_keys AS fk
JOIN 
    sys.foreign_key_columns AS fkc 
    ON fk.object_id = fkc.constraint_object_id
JOIN 
    sys.tables AS tp 
    ON fkc.parent_object_id = tp.object_id
JOIN 
    sys.columns AS cp 
    ON fkc.parent_object_id = cp.object_id AND fkc.parent_column_id = cp.column_id
JOIN 
    sys.tables AS tr 
    ON fkc.referenced_object_id = tr.object_id
JOIN 
    sys.columns AS cr 
    ON fkc.referenced_object_id = cr.object_id AND fkc.referenced_column_id = cr.column_id
join sys.schemas ss on ss.schema_id = fk.schema_id
        """
        result = connection.execute(text(query)).fetchall()
        return result  # Return the result for further use

# Call the function to get the schema and table names
# result = disable_foreign_keys(engine_target)
# print(result)

In [7]:
def nocheck_constraints(engine, result):
    for schema_name, table_name in result:
        try:
            with engine.connect() as connection:
                # Set autocommit to True if needed; otherwise, remove this line
                #connection = connection.execution_options(autocommit=True)
                query = f'ALTER TABLE {schema_name}.{table_name} NOCHECK CONSTRAINT ALL'
                # Log the query for debugging purposes
                print(f"Executing query: {query}")
                connection.execute(text(query).execution_options(autocommit=True)) ##change
                print(f"Constraints disabled for {schema_name}.{table_name}")
        except Exception as e:
            # Print the error for debugging
            print(f"Error disabling constraints for {schema_name}.{table_name}: {e}")

In [8]:
# nocheck_constraints(engine_target, result)

In [9]:
from sqlalchemy.orm import sessionmaker
def delete_table_contents(engine, schema, table_name):
    session = sessionmaker(engine_target)
    with session.begin() as sess:
        delete_statement = f'DELETE FROM {schema}.{table_name}'
        print(delete_statement)
        sess.execute(text(delete_statement))
result = disable_foreign_keys(engine_target)
nocheck_constraints(engine_target, result)
delete_table_contents(engine_target, 'dimension', 'Department')

Executing query: ALTER TABLE fact.Employee NOCHECK CONSTRAINT ALL
Constraints disabled for fact.Employee
Executing query: ALTER TABLE fact.Employee NOCHECK CONSTRAINT ALL
Constraints disabled for fact.Employee
Executing query: ALTER TABLE dimension.EmployeeDepartmentHistory NOCHECK CONSTRAINT ALL
Constraints disabled for dimension.EmployeeDepartmentHistory
Executing query: ALTER TABLE dimension.EmployeeDepartmentHistory NOCHECK CONSTRAINT ALL
Constraints disabled for dimension.EmployeeDepartmentHistory
Executing query: ALTER TABLE dimension.Employee_Payment NOCHECK CONSTRAINT ALL
Constraints disabled for dimension.Employee_Payment
Executing query: ALTER TABLE dimension.Personal_information NOCHECK CONSTRAINT ALL
Constraints disabled for dimension.Personal_information
DELETE FROM dimension.Department


In [10]:
# def delete_table_contents(engine, schema, table_name):
#     with engine.connect() as connection:
#         delete_statement = f'DELETE FROM {schema}.{table_name}'
#         print(delete_statement)
#         connection.execute(text(delete_statement).execution_options(autocommit=True))
#     print(f'Deleted contents of table {schema}.{table_name}')
# # result = disable_foreign_keys(engine_target)
# # nocheck_constraints(engine_target, result)

# # Delete contents from the 'dimension.shift' table
# # delete_table_contents(engine_target, 'dimension', 'shift')

In [11]:
result = disable_foreign_keys(engine_target)
nocheck_constraints(engine_target, result)
delete_table_contents(engine_target, 'dimension', 'Department')

Executing query: ALTER TABLE fact.Employee NOCHECK CONSTRAINT ALL
Constraints disabled for fact.Employee
Executing query: ALTER TABLE fact.Employee NOCHECK CONSTRAINT ALL
Constraints disabled for fact.Employee
Executing query: ALTER TABLE dimension.EmployeeDepartmentHistory NOCHECK CONSTRAINT ALL
Constraints disabled for dimension.EmployeeDepartmentHistory
Executing query: ALTER TABLE dimension.EmployeeDepartmentHistory NOCHECK CONSTRAINT ALL
Constraints disabled for dimension.EmployeeDepartmentHistory
Executing query: ALTER TABLE dimension.Employee_Payment NOCHECK CONSTRAINT ALL
Constraints disabled for dimension.Employee_Payment
Executing query: ALTER TABLE dimension.Personal_information NOCHECK CONSTRAINT ALL
Constraints disabled for dimension.Personal_information
DELETE FROM dimension.Department


In [12]:
def check_table_empty(engine, schema, table_name):
    with engine.connect() as connection:
        query = f'SELECT COUNT(*) FROM {schema}.{table_name}'
        result = connection.execute(text(query)).scalar()
    if result == 0:
        print(f'The table {schema}.{table_name} is empty.')
    else:
        print(f'The table {schema}.{table_name} has {result} rows.')

# Example usage


# delete_table_contents(engine_target, 'dimension', 'department')
check_table_empty(engine_target, 'dimension', 'department')


The table dimension.department is empty.


Extract and load tables

In [13]:
def extract_and_load_employeelogin():
    query = "SELECT BusinessEntityID, LoginID, rowguid FROM HumanResources.Employee"
    df = pd.read_sql(query, engine_source)
    print("Data from Employee loaded into DataFrame:")
   
    delete_table_contents(engine_target, 'dimension', 'employeelogin')
    df.to_sql(name='Employeelogin', con=engine_target, schema='dimension',if_exists = 'append', index=False)

    
extract_and_load_employeelogin()

Data from Employee loaded into DataFrame:
DELETE FROM dimension.employeelogin


In [14]:
# Create an instance of BackgroundScheduler
scheduler = BackgroundScheduler()
scheduler.start()
scheduler.add_job(func=extract_and_load_employeelogin, trigger="interval", minutes=2)


<Job (id=b4180a5fef9a4df2b36cd03b8cea5f78 name=extract_and_load_employeelogin)>

In [15]:
def extract_and_load_department():
    query = "SELECT DISTINCT DepartmentID, Name, GroupName, ModifiedDate FROM HumanResources.Department"
    df = pd.read_sql(query, engine_source)
    print("Data from Department loaded into DataFrame:")
    
  
    delete_table_contents(engine_target, 'dimension', 'Department')
    
    df.to_sql(name='Department', con=engine_target, schema='dimension', if_exists='append', index=False)
    
   
    print('Done!')
    
    send_mail_success(
    subject='ETL Success: Department Transformation',
    message='The data transformation for the Department was successful.'
     )

# Execute the extract and load function
extract_and_load_department()

Data from Department loaded into DataFrame:
DELETE FROM dimension.Department
Done!
Success email sent successfully.


In [16]:
scheduler.add_job(func=extract_and_load_department, trigger="interval", minutes=2)


<Job (id=f426577f8ac94cfcac8b650d06506943 name=extract_and_load_department)>

In [17]:
def extract_load_employee_fact():
    try:
        # Extract Employee data
        df_employee = pd.read_sql(
            """
            SELECT 
                BusinessEntityID
            FROM HumanResources.Employee
            """,
            engine_source
        )
        print("Extracted Employee Data:")
        print(df_employee.head())  # Print the first few rows

        # Extract EmployeeDepartmentHistory data
        df_employee_dept_history = pd.read_sql(
            """
            SELECT 
                BusinessEntityID,
                DepartmentID
            FROM HumanResources.EmployeeDepartmentHistory
            """,
            engine_source
        )
        print("Extracted EmployeeDepartmentHistory Data:")
        print(df_employee_dept_history.head())  # Print the first few rows

        # Merge data
        df_employee_fact = pd.merge(
            df_employee,
            df_employee_dept_history,
            on='BusinessEntityID',
            how='left'
        )
        print("Merged DataFrame:")
        print(df_employee_fact.head())  # Print the first few rows

        # Remove duplicates based on BusinessEntityID
        df_employee_fact = df_employee_fact.drop_duplicates(subset=['BusinessEntityID'])
        print("DataFrame after Removing Duplicates:")
        print(df_employee_fact.head())  # Print the first few rows

        # We do not need to handle EmployeeID, as it will be auto-generated by SQL Server
        df_employee_fact = df_employee_fact[['BusinessEntityID', 'DepartmentID']]

        # Print DataFrame for verification
        print("Final Employee Fact DataFrame:")
        print(df_employee_fact.head())  # Print the first few rows

        # Load data to target table
        target_schema = 'fact'
        target_table = 'Employee'

        # Delete existing contents of the fact.Employee table
        delete_table_contents(engine_target, target_schema, target_table)

        # Insert new data into the fact.Employee table
        df_employee_fact.to_sql(name=target_table, con=engine_target, schema=target_schema, if_exists='append', index=False, chunksize=1000)
        print(f"Data successfully inserted into {target_schema}.{target_table}")
        
                
        send_mail_success(
        subject='ETL Success: Load Employee Fact Transformation',
        message='The data transformation for the Load Employee Fact table was successful.'
        )
    except:
        logger.exception("error test")
        send_mail_error(error = str (traceback.format_exc()),
                    subject = 'Error test')

# Run the ETL process for Employee Fact table
extract_load_employee_fact()


Extracted Employee Data:
   BusinessEntityID
0               263
1                78
2               242
3               125
4               278
Extracted EmployeeDepartmentHistory Data:
   BusinessEntityID  DepartmentID
0                 2             1
1                 3             1
2                 3             2
3                 4             1
4                 4             2
Merged DataFrame:
   BusinessEntityID  DepartmentID
0               263            11
1                78             7
2               242            10
3               125            15
4               278             3
DataFrame after Removing Duplicates:
   BusinessEntityID  DepartmentID
0               263            11
1                78             7
2               242            10
3               125            15
4               278             3
Final Employee Fact DataFrame:
   BusinessEntityID  DepartmentID
0               263            11
1                78             7
2            

In [18]:
scheduler.add_job(func=extract_load_employee_fact, trigger="interval", minutes=2)


<Job (id=719942ffbc8f451880cc196e2128f9ee name=extract_load_employee_fact)>

In [19]:
def extract_and_load_shift():
    query = "SELECT DISTINCT ShiftID, Name, StartTime, EndTime, ModifiedDate FROM HumanResources.Shift"
    df = pd.read_sql(query, engine_source)
    print(f"Data from Shift loaded into DataFrame:")
    print(df.head())
    
    try:
        
        
        # Delete existing records in the target table
        delete_table_contents(engine_target, 'dimension', 'shift')
        
        # Load new data into the target table
        df.to_sql(name='shift', con=engine_target, schema='dimension', if_exists='append', index=False)
        print('Data successfully inserted into dimension.shift')
        
        send_mail_success(
        subject='ETL Success:Shift Transformation',
        message='The data transformation for the Shift table was successful.'
        )
    except:
        logger.exception("error test")
        send_mail_error(error = str (traceback.format_exc()),
                    subject = 'Error test')  

    finally:
        # Enable constraints
       
        print('Constraints re-enabled.')

# Run the ETL process for Shift table
extract_and_load_shift()


Data from Shift loaded into DataFrame:
   ShiftID     Name StartTime   EndTime ModifiedDate
0        1      Day  07:00:00  15:00:00   2008-04-30
1        2  Evening  15:00:00  23:00:00   2008-04-30
2        3    Night  23:00:00  07:00:00   2008-04-30
DELETE FROM dimension.shift
Data successfully inserted into dimension.shift
Success email sent successfully.
Constraints re-enabled.


In [20]:

scheduler.add_job(func=extract_and_load_shift, trigger="interval", minutes=2)


<Job (id=3ac84fd1b5434c5b8c33ce91c983354d name=extract_and_load_shift)>

In [21]:
import pandas as pd
from sqlalchemy import create_engine, text


def timestamp_to_int(ts):
    """Convert Timestamp to an integer format."""
    if pd.isnull(ts):
        return None
    return int.from_bytes(ts.to_pydatetime().timestamp().to_bytes(8, byteorder='little', signed=False), byteorder='big')

def extract_merge_load_employee_department_history():
    with engine_target.connect() as conn:
        # Fetch existing records
        existing_records = pd.read_sql_query("""
            SELECT BusinessEntityID, ShiftID
            FROM dimension.EmployeeDepartmentHistory
            """, conn)
        
        # Convert existing records to a set for quick lookup
        existing_records_set = set(existing_records.itertuples(index=False, name=None))
        
        # Placeholder for loading your new data (replace 'your_dataframe' with actual DataFrame)
        # Example: your_dataframe = pd.read_sql_query("SELECT * FROM source_table", engine_source)
        # Ensure that 'your_dataframe' is defined before this
        your_dataframe = pd.DataFrame()  # Replace with actual code to load new data
        
        # Iterate through your new records
        for _, row in your_dataframe.iterrows():
            business_entity_id = row['BusinessEntityID']
            shift_id = row['ShiftID']
            start_date = row['StartDate']
            modified_date = row['ModifiedDate']
            job_title = row['JobTitle']
            
            # Check for existing record
            if (business_entity_id, shift_id) in existing_records_set:
                print(f"Record with BusinessEntityID={business_entity_id} and ShiftID={shift_id} already exists.")
            else:
                # Insert new record
                insert_query = text("""
                    INSERT INTO dimension.EmployeeDepartmentHistory 
                    (BusinessEntityID, ShiftID, StartDate, EndDate, ModifiedDate, JobTitle, IsCurrent)
                    VALUES (:business_entity_id, :shift_id, :start_date, NULL, :modified_date, :job_title, 1)
                """)
                conn.execute(insert_query, {
                    'business_entity_id': business_entity_id,
                    'shift_id': shift_id,
                    'start_date': start_date,
                    'modified_date': modified_date,
                    'job_title': job_title
                })
        print("Data successfully updated and inserted into dimension.EmployeeDepartmentHistory")
        
        send_mail_success(
        subject='ETL Success: Employee Department History Transformation',
        message='The data transformation for theEmployee Department History table was successful.'
        )

# Call your function
extract_merge_load_employee_department_history()


Data successfully updated and inserted into dimension.EmployeeDepartmentHistory
Success email sent successfully.


In [22]:

scheduler.add_job(func=extract_and_load_shift, trigger="interval", minutes=2)


<Job (id=18ee633a39744f65946c52405866545e name=extract_and_load_shift)>

In [23]:
def extract_transform_load_employee_payment():
    try:
        # Extract EmployeePayHistory data
        df_employee_pay_history = pd.read_sql(
            "SELECT BusinessEntityID, RateChangeDate, Rate AS Salary_hourly_rate, PayFrequency, ModifiedDate FROM HumanResources.EmployeePayHistory",
            engine_source
        )
        print("EmployeePayHistory data loaded into DataFrame:")
        print(df_employee_pay_history.head())

        # Extract Employee data
        df_employee = pd.read_sql(
            "SELECT BusinessEntityID, CurrentFlag, SalariedFlag FROM HumanResources.Employee",
            engine_source
        )
        print("Employee data loaded into DataFrame:")
        print(df_employee.head())

        # Transform data
        df_employee_pay_history['Salary_received_weekly'] = df_employee_pay_history['PayFrequency'].apply(lambda x: 1 if x == 2 else None)
        df_employee_pay_history['Salary_received_monthly'] = df_employee_pay_history['PayFrequency'].apply(lambda x: 1 if x == 1 else None)

        df_merged = pd.merge(
            df_employee_pay_history.drop(columns=['PayFrequency']),
            df_employee,
            on='BusinessEntityID'
        )

        df_merged['Inactive'] = df_merged['CurrentFlag'].apply(lambda x: 1 if x == 0 else None)
        df_merged['Active'] = df_merged['CurrentFlag'].apply(lambda x: 1 if x == 1 else None)
        df_merged['Hourly'] = df_merged['SalariedFlag'].apply(lambda x: 1 if x == 0 else None)
        df_merged['Salaried'] = df_merged['SalariedFlag'].apply(lambda x: 1 if x == 1 else None)

        df_final = df_merged[[
            'BusinessEntityID',
            'RateChangeDate',
            'Salary_hourly_rate',
            'Salary_received_monthly',
            'Salary_received_weekly',
            'ModifiedDate',
            'Inactive',
            'Active',
            'Hourly',
            'Salaried'
        ]]
        print("Transformed DataFrame:")
        print(df_final.head())

        # Load data to target table
        target_schema = 'dimension'
        target_table = 'Employee_Payment'

     
        delete_table_contents(engine_target, target_schema, target_table)
        check_table_empty(engine_target, target_schema, target_table)

        df_final.to_sql(name=target_table, con=engine_target, schema=target_schema, if_exists='append', index=False, chunksize=1000)
        print("Data successfully inserted into dimension.Employee_Payment")
        
        # Send email after successful transformation
        send_mail_success(
            subject='ETL Success: Employee Payment Transformation',
            message='The data transformation for the Employee Payment table was successful.'
        )

    except:
        logger.exception("error test")
        send_mail_error(error = str (traceback.format_exc()),
                    subject = 'Error test')

  

# Run the ETL process for Employee_Payment table
extract_transform_load_employee_payment()


EmployeePayHistory data loaded into DataFrame:
   BusinessEntityID RateChangeDate  Salary_hourly_rate  PayFrequency  \
0                 1     2009-01-14            125.5000             1   
1                 2     2008-01-31             63.4615             2   
2                 3     2007-11-11             43.2692             2   
3                 4     2007-12-05              8.6200             2   
4                 4     2010-05-31             23.7200             2   

  ModifiedDate  
0   2014-06-30  
1   2014-06-30  
2   2014-06-30  
3   2007-11-21  
4   2010-05-16  
Employee data loaded into DataFrame:
   BusinessEntityID  CurrentFlag  SalariedFlag
0                 1         True          True
1                 2         True          True
2                 3         True          True
3                 4         True         False
4                 5         True          True
Transformed DataFrame:
   BusinessEntityID RateChangeDate  Salary_hourly_rate  \
0                 

In [24]:
scheduler.add_job(func=extract_transform_load_employee_payment, trigger="interval", minutes=2)


<Job (id=27f6d4941d8f493fa2d3054e48f37eb6 name=extract_transform_load_employee_payment)>

In [25]:
def extract_merge_load_employee_department_history():
    # Extract JobTitle and BusinessEntityID from Employee table
    query_employee = """
    SELECT BusinessEntityID, JobTitle
    FROM HumanResources.Employee
    """
    try:
        df_employee = pd.read_sql(query_employee, engine_source)
        print("Employee data loaded into DataFrame:")
        print(df_employee.head())
    except Exception as e:
        print(f"Error loading employee data: {e}")
        return

    # Extract EmployeeDepartmentHistory data
    query_history = """
    SELECT BusinessEntityID, ShiftID, StartDate, EndDate, ModifiedDate
    FROM HumanResources.EmployeeDepartmentHistory
    """
    try:
        df_history = pd.read_sql(query_history, engine_source)
        print("EmployeeDepartmentHistory data loaded into DataFrame:")
        print(df_history.head())
    except Exception as e:
        print(f"Error loading employee department history data: {e}")
        return

    # Merge DataFrames on BusinessEntityID
    df_merged = pd.merge(df_history, df_employee, on='BusinessEntityID', how='left')

    print("Merged DataFrame:")
    print(df_merged.head())

    # Load data to target table
    target_schema = 'dimension'
    target_table = 'EmployeeDepartmentHistory'

    try:
        delete_table_contents(engine_target, target_schema, target_table)
        check_table_empty(engine_target, target_schema, target_table)

        df_merged.to_sql(target_table, con=engine_target, schema=target_schema, if_exists='append', index=False, chunksize=1000)
        print("Data successfully inserted into dimension.EmployeeDepartmentHistory")
        send_mail_success(
        subject='ETL Success: Employee Department History Transformation',
        message='The data transformation for the Employee Department History table was successful.'
        )

    except:
        logger.exception("error test")
        send_mail_error(error = str (traceback.format_exc()),
                    subject = 'Error test')

      

# Run the ETL process for EmployeeDepartmentHistory table
extract_merge_load_employee_department_history()

Employee data loaded into DataFrame:
   BusinessEntityID                       JobTitle
0                 1        Chief Executive Officer
1                 2  Vice President of Engineering
2                 3            Engineering Manager
3                 4           Senior Tool Designer
4                 5                Design Engineer
EmployeeDepartmentHistory data loaded into DataFrame:
   BusinessEntityID  ShiftID   StartDate     EndDate            ModifiedDate
0                 1        2  2009-01-14  2024-08-17 2024-08-17 10:36:30.557
1                 1        2  2024-08-12  2024-08-17 2024-08-17 10:36:30.557
2                 1        2  2024-08-12  2024-08-17 2024-08-17 10:36:30.557
3                 2        1  2008-01-31        None 2008-01-30 00:00:00.000
4                 2        2  2020-08-12        None 2024-08-12 13:38:03.163
Merged DataFrame:
   BusinessEntityID  ShiftID   StartDate     EndDate            ModifiedDate  \
0                 1        2  2009-01-14  2

In [26]:
scheduler.add_job(func=extract_merge_load_employee_department_history, trigger="interval", minutes=2)


<Job (id=9e51fd438ee74b1391ad2a603ffe2b2a name=extract_merge_load_employee_department_history)>

In [27]:
def extract_load_personal_information():
    try:
        # Extract Employee information
        df_employee_info = pd.read_sql(
            """
            SELECT 
                BusinessEntityID,
                NationalIDNumber,
                BirthDate,
                MaritalStatus,
                Gender,
                HireDate,
                VacationHours,
                SickLeaveHours,
                CAST(OrganizationNode AS VARCHAR(100)) AS employee_location_in_corporate_hierarchy,
                CAST(OrganizationLevel AS VARCHAR(100)) As employee_depth_in_corporate_hierarchy
            FROM HumanResources.Employee
            """,
            engine_source
        )
        print("Employee information loaded into DataFrame:")
        print(df_employee_info.head())

        # Extract JobCandidate information
        df_job_candidate = pd.read_sql(
            """
            SELECT 
                BusinessEntityID,
                Resume
            FROM HumanResources.JobCandidate
            """,
            engine_source
        )
        print("JobCandidate information loaded into DataFrame:")
        print(df_job_candidate.head())

        # Merge data
        df_personal_info = pd.merge(
            df_employee_info,
            df_job_candidate,
            on='BusinessEntityID',
            how='left'
        )
        print("Merged DataFrame:")
        print(df_personal_info.head())

        # Drop the computed column to avoid errors during insertion
        # df_personal_info = df_personal_info.drop(columns=['employee_depth_in_corporate_hierarchy'])

        # Load data to target table
        target_schema = 'dimension'
        target_table = 'Personal_information'

        delete_table_contents(engine_target, target_schema, target_table)
        check_table_empty(engine_target, target_schema, target_table)
        df_personal_info = df_personal_info[['BusinessEntityID', 'NationalIDNumber', 'BirthDate', 'MaritalStatus',
       'Gender', 'HireDate', 'VacationHours', 'SickLeaveHours','Resume',
       'employee_location_in_corporate_hierarchy' ,'employee_depth_in_corporate_hierarchy']]

        df_personal_info.to_sql(
            name=target_table,
            con=engine_target,
            schema=target_schema,
            if_exists='append',
            index=False,
            chunksize=1000
        )
        print("Data successfully inserted into dimension.Personal_information")
        
        send_mail_success(
        subject='ETL Success: Personal information Transformation',
        message='The data transformation for the Personal information  History table was successful.'
        )

    except:
        logger.exception("error test")
        send_mail_error(error = str (traceback.format_exc()),
                    subject = 'Error test')


# Run the ETL process for Personal_information table
extract_load_personal_information()


Employee information loaded into DataFrame:
   BusinessEntityID NationalIDNumber   BirthDate MaritalStatus Gender  \
0                 1        295847284  1969-01-29             S      M   
1                 2        245797967  1971-08-01             S      F   
2                 3        509647174  1974-11-12             M      M   
3                 4        112457891  1974-12-23             S      M   
4                 5        695256908  1952-09-27             M      F   

     HireDate  VacationHours  SickLeaveHours  \
0  2009-01-14             99              69   
1  2008-01-31              1              20   
2  2007-11-11              2              21   
3  2007-12-05             48              80   
4  2008-01-06              5              22   

  employee_location_in_corporate_hierarchy  \
0                                     None   
1                                      /1/   
2                                    /1/1/   
3                                  /1/1/1/  

In [28]:
scheduler.add_job(func=extract_load_personal_information, trigger="interval", minutes=2)


<Job (id=e50a520012c14b82ba7a3b8c7a979ba6 name=extract_load_personal_information)>

In [29]:

# try:
#     # Extract Employee information
#     df_employee_info = pd.read_sql(
#         """
#         SELECT 
#             BusinessEntityID,
#             NationalIDNumber,
#             BirthDate,
#             MaritalStatus,
#             Gender,
#             HireDate,
#             VacationHours,
#             SickLeaveHours,
#             CAST(OrganizationNode AS VARCHAR(100)) AS employee_location_in_corporate_hierarchy
#         FROM HumanResources.Employee
#         """,
#         engine_source
#     )
#     # print("Employee information loaded into DataFrame:")
#     # print(df_employee_info.head())

#     # Extract JobCandidate information
#     df_job_candidate = pd.read_sql(
#         """
#         SELECT 
#             BusinessEntityID,
#             Resume
#         FROM HumanResources.JobCandidate
#         """,
#         engine_source
#     )
#     #print("JobCandidate information loaded into DataFrame:")
#     #print(df_job_candidate.head())

#     # Merge data
#     df_personal_info = pd.merge(
#         df_employee_info,
#         df_job_candidate,
#         on='BusinessEntityID',
#         how='left'
#     )
#     #print("Merged DataFrame:")
#     print(df_personal_info.head())
# except Exception as e:
#     print(f"Error during ETL process for Personal_information: {e}")



In [30]:
#  df_personal_info.head()
# df_personal_info[['BusinessEntityID', 'NationalIDNumber', 'BirthDate', 'MaritalStatus',
#        'Gender', 'HireDate', 'VacationHours', 'SickLeaveHours','Resume',
#        'employee_location_in_corporate_hierarchy' ]]

In [31]:
for  schema_name, table_name in result:
    with engine_target.connect() as connection:
        query = f'alter table {schema_name}.{table_name} check constraint all'
        connection.execute(text(query).execution_options(autocommit=True))
    

In [32]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

def send_mail_success(
    sender="osamamenna1511@gmail.com",  # put sender email here 
    password='lwrx jrtd uhjm cjvt',  # use your app-specific password here
    to_list=["menna1eminshawy@gmail.com"],  # receiver list here
    message="Data transformation completed successfully.",
    subject="Data Transformation Success",
):
    msg = MIMEMultipart()
    msg["From"] = sender
    msg["To"] = ", ".join(to_list)
    msg["Subject"] = subject
    html_message = f"""<html><body>{message}</body></html>"""
    msg.attach(MIMEText(html_message, "html"))
    
    try:
        # sends email
        smtpserver = smtplib.SMTP("smtp.gmail.com", 587)
        smtpserver.starttls()
        smtpserver.login(sender, password)
        smtpserver.sendmail(msg["From"], to_list, msg.as_string())
        smtpserver.quit()
        print("Success email sent successfully.")
    except Exception as e:
        print(f"Failed to send success email: {e}")

# Example usage after data transformation is done
send_mail_success(
    message="The data transformation process has been completed successfully and is ready for further steps."
)


Success email sent successfully.
